In [1]:
from mmdet.apis import init_detector, inference_detector, show_result_pyplot
import mmcv
from mmcv import Config


import copy
import os.path as osp

import numpy as np

from mmdet.datasets.builder import DATASETS
from mmdet.datasets.custom import CustomDataset

from mmdet.apis import set_random_seed

from mmdet.core.post_processing import bbox_nms


import base64

import glob as _glob
import os

import json
from io import BytesIO
from PIL import Image
import pandas as pd 

from mmcv.visualization import image

In [2]:
def glob(dir, pats, recursive=False):  # faster than match, python3 only
    pats = pats if isinstance(pats, (list, tuple)) else [pats]
    matches = []
    for pat in pats:
        matches += _glob.glob(os.path.join(dir, pat), recursive=recursive)
    return matches

In [3]:
## 추가수정 기존 받았던 pretrain과 매칭되는 config로 수정 
cfg = Config.fromfile('mmdetection/configs/resnest/cascade_rcnn_s101_fpn_syncbn-backbone+head_mstrain-range_1x_coco.py') 

In [4]:
@DATASETS.register_module()
class lesion_ds(CustomDataset):
    CLASSES=('01_ulcer','02_mass','04_lymph', '05_bleeding')


    def load_annotations(self, ann_file):
        
        CLASSES_dict = { '01_ulcer' : 0 , '02_mass' : 1, '04_lymph' : 2, '05_bleeding' : 3}
        
        # load image list from file
        image_list = mmcv.list_from_file(self.ann_file)
        
        data_infos = []
        
        for idx,img in enumerate(image_list):
            json_data = {}
            with open(img, "r") as json_file:
                json_data = json.load(json_file)
            
            filename = img # json에 annotation + image라서 json 자체를 filename로 주고 LoadImageFromFile을 baseline을 참조하여 custom으로 

            height = json_data['imageHeight']
            width = json_data['imageWidth']

            data_info = dict(filename=filename, width=width, height=height)

            gt_bboxes = []
            gt_labels = []

            for a_idx in range(len(json_data['shapes'])):
                gt_labels.append(CLASSES_dict[json_data['shapes'][a_idx]['label']])
                
                ## 좌표순서 좌상 우상 우하 좌하 
                ## mmdetection의 default annotation loader는 'coco_panoptic.py'에 있는데 여기 기준으로 하면 bbox에는 x1,y1,x2,y2가 담겨야함
                ## 만약에 이렇게 따로 annotation을 load를 하지 않을시에는 x,y,w,h 형태로 annotaion을 생성하면 자동으로 x1,y1,x2,y2로 변환해줌 
                ori_pos = np.array(json_data['shapes'][a_idx]['points'])
                x1, y1, x2, y2 = min(ori_pos[:, 0]), min(ori_pos[:, 1]), max(ori_pos[:, 0]), max(ori_pos[:, 1])
                
                if x1 == 0 : 
                    x1 = 1 
                if y1 == 0 : 
                    y1 = 1 
                
                if x2 == width: 
                    x2 = x2 - 1 
                if y2 == width: 
                    y2 = y2 - 1 
            
                
                if x1==x2 or y1==y2:
                    print('Grond-truth Bounding Box 이상체크')
                    print(filename)
                     
                
                gt_bboxes.append([x1,y1,x2,y2])
                

            data_anno = dict(
                    bboxes=np.array(gt_bboxes, dtype=np.float32).reshape(-1, 4),
                    labels=np.array(gt_labels, dtype=np.long))


            data_info.update(ann=data_anno)
            data_infos.append(data_info)
            
            if idx!=0 and idx%20000==0:
                print(str(idx)+'/'+str(len(image_list))+' load annotations END!')
            
        
        
        return data_infos

In [5]:
#sync bn하면 single gpu라 error 발생 
cfg.norm_cfg = dict(type='BN', requires_grad=True)

#class 4개로 
cfg.model.roi_head.bbox_head[0].num_classes=4

cfg.model.roi_head.bbox_head[1].num_classes=4

cfg.model.roi_head.bbox_head[2].num_classes=4

#데이터 pipeline은 config 참조 
cfg.img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)

## 실제 test시에는 Test Time Augmentation 적용하는 것으로 
cfg.test_pipeline = [
    dict(type='CUSTOM_LoadImageFromFile'),
    dict(
        type='MultiScaleFlipAug',
        img_scale=(600, 600),
        flip=False,
        transforms=[
            dict(type='Resize', keep_ratio=True),
            dict(type='RandomFlip'),
            dict(type='Normalize', **cfg.img_norm_cfg),
            dict(type='Pad', size_divisor=32),
            dict(type='ImageToTensor', keys=['img']),
            dict(type='Collect', keys=['img'])
        ])
]
cfg.data = dict(
    test=dict(pipeline=cfg.test_pipeline))


cfg.data.test.type = cfg.dataset_type


# seed set
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = [0]

print(f'Config:\n{cfg.pretty_text}')

Config:
model = dict(
    type='CascadeRCNN',
    backbone=dict(
        type='ResNeSt',
        depth=101,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=True),
        norm_eval=False,
        style='pytorch',
        init_cfg=dict(type='Pretrained', checkpoint='open-mmlab://resnest101'),
        stem_channels=128,
        radix=2,
        reduction_factor=4,
        avg_down_stride=True),
    neck=dict(
        type='FPN',
        in_channels=[256, 512, 1024, 2048],
        out_channels=256,
        num_outs=5),
    rpn_head=dict(
        type='RPNHead',
        in_channels=256,
        feat_channels=256,
        anchor_generator=dict(
            type='AnchorGenerator',
            scales=[8],
            ratios=[0.5, 1.0, 2.0],
            strides=[4, 8, 16, 32, 64]),
        bbox_coder=dict(
            type='DeltaXYWHBBoxCoder',
            target_means=[0.0, 0.0, 0.0, 0.0],
            target_stds=

In [6]:
# Build the detector
#model = init_detector(config, checkpoint, device='cuda:0')
checkpoint='lesion_checkpoints_ver2/epoch_10.pth'
# model = init_detector(cfg,checkpoint, device='cpu')
model = init_detector(cfg, checkpoint, device='cuda:0')
model = model.eval()

load checkpoint from local path: lesion_checkpoints_ver2/epoch_10.pth


In [7]:
test_ls=glob('lesion_DB/test','*')
test_ls.sort()

In [10]:
CLASSES_dict = { 0 : '01_ulcer' , 1 : '02_mass' , 2 : '04_lymph' , 3 : '05_bleeding' }
CLASS_NAME=['01_ulcer','02_mass','04_lymph','05_bleeding']

results_for_csv = {
    'file_name':[], 'class_id':[], 'confidence':[], 'point1_x':[], 'point1_y':[],
    'point2_x':[], 'point2_y':[], 'point3_x':[], 'point3_y':[], 'point4_x':[], 'point4_y':[]
}


### 기존 NMS사용시 단순 Threshoold 기준으로 output 간추려 낸것보다 성능이 낮아짐 
### https://github.com/BichenWuUCB/squeezeDet 소스 사용 
def iou(box1, box2):
  '''Compute the Intersection-Over-Union of two given boxes.
  Args:
    box1: array of 4 elements [cx, cy, width, height].
    box2: same as above
  Returns:
    iou: a float number in range [0, 1]. iou of the two boxes.
  '''

  lr = min(box1[0]+0.5*box1[2], box2[0]+0.5*box2[2]) - \
      max(box1[0]-0.5*box1[2], box2[0]-0.5*box2[2])
  if lr > 0:
    tb = min(box1[1]+0.5*box1[3], box2[1]+0.5*box2[3]) - \
        max(box1[1]-0.5*box1[3], box2[1]-0.5*box2[3])
    if tb > 0:
      intersection = tb*lr
      union = box1[2]*box1[3]+box2[2]*box2[3]-intersection

      return intersection/union

  return 0

def batch_iou(boxes, box):
  '''Compute the Intersection-Over-Union of a batch of boxes with another
  box.
  Args:
    box1: 2D array of [cx, cy, width, height].
    box2: a single array of [cx, cy, width, height]
  Returns:
    ious: array of a float number in range [0, 1].
  '''
  lr = np.maximum(
      np.minimum(boxes[:,0]+0.5*boxes[:,2], box[0]+0.5*box[2]) - \
      np.maximum(boxes[:,0]-0.5*boxes[:,2], box[0]-0.5*box[2]),
      0
  )
  tb = np.maximum(
      np.minimum(boxes[:,1]+0.5*boxes[:,3], box[1]+0.5*box[3]) - \
      np.maximum(boxes[:,1]-0.5*boxes[:,3], box[1]-0.5*box[3]),
      0
  )
  inter = lr*tb
  union = boxes[:,2]*boxes[:,3] + box[2]*box[3] - inter
  return inter/union

def nms(boxes, probs, threshold):
  '''Non-Maximum supression.
  Args:
    boxes: array of [cx, cy, w, h] (center format)
    probs: array of probabilities
    threshold: two boxes are considered overlapping if their IOU is largher than
        this threshold
    form: 'center' or 'diagonal'
  Returns:
    keep: array of True or False.
  '''
 
  order = probs.argsort()[::-1]
  keep = [True]*len(order)
 
  for i in range(len(order)-1):
    ovps = batch_iou(boxes[order[i+1:]], boxes[order[i]])
    for j, ov in enumerate(ovps):
      if ov > threshold:
        keep[order[j+i+1]] = False
  return keep

### https://github.com/BichenWuUCB/squeezeDet 소스 사용 

for t_idx, filename in enumerate(test_ls):
    with open(filename, "r") as json_file:
            json_data = json.load(json_file)

    img_bytes = base64.b64decode(json_data['imageData'])

    img = mmcv.imfrombytes(img_bytes, flag='color',backend='pillow')

    img = img.astype(np.float32)
    
    
    ## output shape x1, y1, x2, y2 , confidence score
    results = inference_detector(model, img)
    
    ## 단순 threshold 값 기준으로 하면 겹치는 box가 너무많이 생겨서 발생하는 문제 해결이 안되기 때문에 NMS로 겹쳐지는 박스의 경우 제거 해주는 것이 필요해보임 
    for cls_idx, result in enumerate(results):
        if result.shape[0]!=0:
            out_nms = nms(result[:,0:4],result[:,4],0.4)
            save_target = result[out_nms]
            for pos_idx, pos, in enumerate(save_target):
                results_for_csv['file_name'].append(filename.split('/')[-1])
                results_for_csv['class_id'].append(cls_idx+1)
                results_for_csv['confidence'].append(pos[4])
                results_for_csv['point1_x'].append(pos[0])
                results_for_csv['point1_y'].append(pos[1])
                results_for_csv['point2_x'].append(pos[2])
                results_for_csv['point2_y'].append(pos[1])
                results_for_csv['point3_x'].append(pos[2])
                results_for_csv['point3_y'].append(pos[3])
                results_for_csv['point4_x'].append(pos[0])
                results_for_csv['point4_y'].append(pos[3])
    
    
    ## 적정 threshold 값 모르겠으니 일단 test output 싹다 저장해서 이미지 봐도 어느정도 score가 좋을지 잘모르겠음..
    # model.show_result(img,results,score_thr=0.0,out_file='infer_show_checkpoints10/'+filename.split('/')[-1][:-5]+'.png')
    
    
    
    
    
    ## result list로 0 1 2 3  ==> 4개 class 
    ## class id 는 1부터 시작하니까 +1 해줌 
    #  0.407    0.89
    # for cls_idx, result in enumerate(results):
    #     ## output이 0이 아니고 
    #     if result.shape[0]!=0:
    #         # print(result)
    #         thre_val = result[np.where(result[:,4]>=0.407)]
    #         if thre_val.shape[0]!=0:
    #             for pos_idx, pos, in enumerate(thre_val):
    #                 results_for_csv['file_name'].append(filename.split('/')[-1])
    #                 results_for_csv['class_id'].append(cls_idx+1)
    #                 results_for_csv['confidence'].append(pos[4])
    #                 results_for_csv['point1_x'].append(pos[0])
    #                 results_for_csv['point1_y'].append(pos[1])
    #                 results_for_csv['point2_x'].append(pos[2])
    #                 results_for_csv['point2_y'].append(pos[1])
    #                 results_for_csv['point3_x'].append(pos[2])
    #                 results_for_csv['point3_y'].append(pos[3])
    #                 results_for_csv['point4_x'].append(pos[0])
    #                 results_for_csv['point4_y'].append(pos[3])
            #confidence score가 0.5보다 클 때
            # if result[np.where(result[:,4]>0.5)].shape[0]!=0:
            #         print('11')
    if t_idx%100==0:
        print(str(t_idx) + ' end!')
     
            

0 end!
100 end!
200 end!
300 end!
400 end!
500 end!
600 end!
700 end!
800 end!
900 end!
1000 end!
1100 end!
1200 end!
1300 end!
1400 end!
1500 end!
1600 end!
1700 end!
1800 end!
1900 end!
2000 end!
2100 end!
2200 end!
2300 end!
2400 end!
2500 end!
2600 end!
2700 end!
2800 end!
2900 end!
3000 end!
3100 end!
3200 end!
3300 end!
3400 end!
3500 end!
3600 end!
3700 end!
3800 end!
3900 end!
4000 end!
4100 end!
4200 end!
4300 end!
4400 end!
4500 end!
4600 end!
4700 end!
4800 end!
4900 end!
5000 end!
5100 end!
5200 end!
5300 end!
5400 end!
5500 end!
5600 end!
5700 end!
5800 end!
5900 end!
6000 end!
6100 end!
6200 end!
6300 end!
6400 end!
6500 end!
6600 end!
6700 end!
6800 end!
6900 end!
7000 end!
7100 end!
7200 end!
7300 end!
7400 end!
7500 end!
7600 end!
7700 end!
7800 end!
7900 end!
8000 end!
8100 end!
8200 end!
8300 end!
8400 end!
8500 end!
8600 end!
8700 end!
8800 end!
8900 end!
9000 end!
9100 end!
9200 end!
9300 end!
9400 end!
9500 end!
9600 end!
9700 end!
9800 end!
9900 end!
10000 end!
1

In [11]:
submission = pd.DataFrame(results_for_csv)
submission.to_csv('HJS_VER1_NMS_TH_IOU0.4.csv', index=False)